# Task 5

**Stanford Research Institute Problem Solver (STRIPS)**  
STRIPS is a formal representation framework for defining planning problems, originally developed to manage a robot’s behavior within a controllable, physical environment. Its main focus is on the automated construction of plans; ordered sequences of actions that transform a system from an initial configuration into a specified goal state. [^STRIPS-in-AI]

**Key Components of STRIPS:**  
    1. **States:** Represented as collections of logical propositions that describe the current situation.  
    2. **Goals:** Expressed as a set of conditions that must hold true to consider the task successfully completed.  
    3. **Actions:** In STRIPS, each action is defined by three key parts:  
         - **Preconditions:** Statements that must be satisfied before the action can be applied.  
         - **Add Effects:** Conditions that become true as a result of executing the action.  
         - **Delete Effects:** Conditions that become false as a result of executing the action. [^STRIPS-in-AI] [^wiki]

**Terms:**
1. **Heuristics:** Techniques used to guide problem-solving by reducing the search space and improving efficiency. While heuristics do not guarantee optimal solutions, they often produce good or optimal results in practical timeframes. [^STRIPS-in-AI]
2. **Symbols:** Abstract representations used to encode knowledge in a form that AI systems can process and reason about, enabling the modeling of relationships and states within the planning domain. [^STRIPS-in-AI]

### Planning with STRIPS
1. **Define the Initial State:** Where the system starts.
2. **Set the Goal State:** What the system should achieve.
3. **Develop Actions:** Defined by their preconditions and effects.
4. **Search for Solutions:** Using a strategy like backward chaining from the goal state to the initial state, identifying actions that satisfy the goal conditions. [^STRIPS-in-AI]

### Conclusion
While STRIPS was revolutionary, it has limitations, primarily its assumption of a static world and the lack of support for actions with nondeterministic outcomes or concurrent actions. Newer models and languages have built upon and extended its original framework, the basic principles of STRIPS continue to influence the field of AI. [^STRIPS-in-AI]

### Given task
Use A* search to find logical operations that change the state and lead to the desired final outcome.

### References
[^STRIPS-in-AI]: Webpage, [STRIPS in AI](https://www.geeksforgeeks.org/artificial-intelligence/strips-in-ai/) - GeeksforGeeks  
[^wiki]: Webpage, [Stanford Research Institute Problem Solver](https://en.wikipedia.org/wiki/Stanford_Research_Institute_Problem_Solver) - Wikipedia  
[^GOAP]: Webpage, [Goal-Oriented Action Planning (GOAP)](https://static.hlt.bme.hu/semantics/external/pages/GOAP/alumni.media.mit.edu/_jorkin/goap.html) - a simplfied STRIPS-like planning architecture  
[Shakey the Robot](https://web.eecs.umich.edu/~stellayu/teach/2023action/papers/2017kuipersShakey.pdf) - First "intelligent" robot (1966-1972) with STRIPS planning. Robot had heuristic search, A*, computer vision and much more!  
ChatGPT assistance used for [How it works](#How-it-works) section, adding comments, streamlining code and creating testing examples for algorithm.

### Shortcuts to code
[1. PriorityQueue](#PriorityQueue)  
[2. Action](#Action)  
[3. GOAPPlanner](#GOAP-planner)  
[4. Helper functions](#Helper-functions)  

[Case 1 Monkey-and-Bananas](#Case1-Monkey-and-Bananas)  
[Case 2 Shakey The Robot](#Case2-Shakey-the-robot)  
[Case 3 Game AI](#Case3-Game-AI)  
[Case 4 Dynamic action generation](#Case4-Dynamic-action-generation)  

[run cases](#main-run)

[How it works](#How-it-works)

## PriorityQueue

In [1]:
class PriorityQueue:    
    def __init__(self):
        # Store (priority, value) tuples
        self.Queue = []
    
    def Enqueue(self, value, priority):
        # Find the correct position and insert
        # Higher priority values come first
        position = 0
        # Find correct position (lower priority = earlier)
        while position < len(self.Queue) and self.Queue[position][0] <= priority:
            position = position + 1
        self.Queue.insert(position, (priority, value))
    
    def Dequeue(self):
        # Remove and return the first item (highest priority)
        if len(self.Queue) == 0:
            raise Exception("Priority queue is empty")
        item = self.Queue.pop(0)
        return item[1]
    
    def Peek(self):
        # View the lowest priority value without removing it.
        if len(self.Queue) == 0:
            raise Exception("Priority queue is empty")
        return self.Queue[0][1]
    
    def IsEmpty(self):
        # Check if queue is empty.
        return len(self.Queue) == 0
    
    def Contains(self, value):
        # Check if value exists in queue.
        for item in self.Queue:
            if item[1] == value:
                return True
        return False
    
    def Remove(self, value):
        # Remove a specific value from queue.
        for i, item in enumerate(self.Queue):
            if item[1] == value:
                self.Queue.pop(i)
                return True
        return False

## Action

In [2]:
class Action:
    """
    STRIPS action definition.
    
    An action consists of:
    - name: Action identifier
    - preconditions: Conditions that must be true before execution
    - add_effects: Conditions that become true after execution
    - delete_effects: Conditions that become false after execution
    - cost: Action cost (used in A* search)
    
    Here is example how this class could work:
        move = Action(
            name="Move(A,B)",
            preconditions={"At(A)"},
            add_effects={"At(B)"},
            delete_effects={"At(A)"},
            cost=1
        )
    """
    
    def __init__(self, name, preconditions=None, add_effects=None, 
                 delete_effects=None, cost=1):
        self.name = name
        self.preconditions = set(preconditions) if preconditions else set()
        self.add_effects = set(add_effects) if add_effects else set()
        self.delete_effects = set(delete_effects) if delete_effects else set()
        self.cost = cost
    
    def is_applicable(self, state):
        # Check if action can be executed in given state.
        # All preconditions must be present in state.
        return self.preconditions.issubset(state)
    
    def apply(self, state):
        # Execute action and return new state.
        # New state = (old state - delete_effects) + add_effects 
        # Returns None if preconditions are not met.
        if not self.is_applicable(state):
            return None
        
        new_state = set(state)
        new_state -= self.delete_effects
        new_state |= self.add_effects
        return frozenset(new_state)
    
    def __repr__(self):
        return f"Action({self.name}, cost={self.cost})"

## GOAP-planner

In [3]:
class GOAPPlanner:
    """
    Goal-Oriented Action Planner using A* search.
    
    Finds the optimal (lowest cost) sequence of actions
    that transforms initial state into goal state.
    
    Usage:
        planner = GOAPPlanner(actions_list)
        plan, cost = planner.plan(initial_state, goal_state)
    """
    
    def __init__(self, actions):
        # Initialize planner with list of available actions.
        self.actions = actions
    
    def heuristic(self, state, goal):
        # Heuristic function: Count of goal conditions not yet satisfied.
        missing = goal - state
        return len(missing)
    
    def get_applicable_actions(self, state):
        # Return list of actions that can be executed in given state.
        return [action for action in self.actions if action.is_applicable(state)]
    
    def plan(self, initial_state, goal_state):
        """
        Find optimal action sequence using A* algorithm.
        
        Args:
            initial_state: Starting state (set of conditions)
            goal_state: Target state (set of conditions)
        
        Returns:
            tuple: (actions_list, total_cost) or (None, None) if no plan found
        """
        # Convert states to frozenset (hashable for dict keys)
        start = frozenset(initial_state)
        goal = frozenset(goal_state)
        
        # A* data structures
        open_list = PriorityQueue()
        closed_set = set()
        
        # g_score: cost from start to each state
        g_score = {start: 0}
        
        # f_score: g + heuristic
        f_score = {start: self.heuristic(start, goal)}
        
        # Track path: state -> (previous_state, action)
        came_from = {}
        
        # Add start state to open list
        open_list.Enqueue(start, f_score[start])
        
        while not open_list.IsEmpty():
            # Get state with lowest f-score
            current = open_list.Dequeue()
            
            # Check if goal is reached (all goal conditions are in state)
            if goal.issubset(current):
                return self._reconstruct_plan(came_from, current)
            
            closed_set.add(current)
            
            # Try all applicable actions
            for action in self.get_applicable_actions(current):
                # Apply action to get new state
                next_state = action.apply(current)
                
                if next_state is None or next_state in closed_set:
                    continue
                
                # Calculate new g-score
                tentative_g = g_score[current] + action.cost
                
                # If found better path or new state
                if next_state not in g_score or tentative_g < g_score[next_state]:
                    came_from[next_state] = (current, action)
                    g_score[next_state] = tentative_g
                    f = tentative_g + self.heuristic(next_state, goal)
                    f_score[next_state] = f
                    
                    if not open_list.Contains(next_state):
                        open_list.Enqueue(next_state, f)
        
        # No plan found
        return None, None
    
    def _reconstruct_plan(self, came_from, final_state):
        # Reconstruct action sequence backward.
        actions = []
        total_cost = 0
        current = final_state
        
        while current in came_from:
            prev_state, action = came_from[current]
            actions.append(action)
            total_cost += action.cost
            current = prev_state
        
        actions.reverse()
        return actions, total_cost

## Helper-functions

In [4]:
def print_plan(plan, cost, initial_state):
    # Print a plan with step-by-step state changes.
    if plan is None:
        print("No plan found!")
        return
    
    print(f"\nPlan found! Total cost: {cost}")
    print(f"Number of actions: {len(plan)}")
    print("\nActions:")
    
    current_state = set(initial_state)
    for i, action in enumerate(plan, 1):
        print(f"  {i}. {action.name} (cost: {action.cost})")
        current_state = set(action.apply(current_state))
    
    print(f"\nFinal state: {current_state}")

## Case1-Monkey-and-Bananas

In [5]:
def example_monkey_and_bananas():
    """
    Classic STRIPS example: Monkey and Bananas problem.
    
    Scenario:
    - Monkey is at location A
    - Box is at location C
    - Bananas hang from ceiling at location B
    - Monkey needs to push box to B, climb up, and grab bananas
    """
    print("\nEXAMPLE 1: Monkey and Bananas")
    
    actions = [
        # Movement actions (when at ground level)
        Action("Move(A->B)", {"At(A)", "Level(low)"}, {"At(B)"}, {"At(A)"}, 1),
        Action("Move(A->C)", {"At(A)", "Level(low)"}, {"At(C)"}, {"At(A)"}, 1),
        Action("Move(B->A)", {"At(B)", "Level(low)"}, {"At(A)"}, {"At(B)"}, 1),
        Action("Move(B->C)", {"At(B)", "Level(low)"}, {"At(C)"}, {"At(B)"}, 1),
        Action("Move(C->A)", {"At(C)", "Level(low)"}, {"At(A)"}, {"At(C)"}, 1),
        Action("Move(C->B)", {"At(C)", "Level(low)"}, {"At(B)"}, {"At(C)"}, 1),
        
        # Push box (monkey and box must be at same location)
        Action("PushBox(C->B)", {"At(C)", "BoxAt(C)", "Level(low)"}, 
               {"At(B)", "BoxAt(B)"}, {"At(C)", "BoxAt(C)"}, 2),
        Action("PushBox(C->A)", {"At(C)", "BoxAt(C)", "Level(low)"}, 
               {"At(A)", "BoxAt(A)"}, {"At(C)", "BoxAt(C)"}, 2),
        
        # Climb up on box (must be at same location as box)
        Action("ClimbUp", {"At(B)", "BoxAt(B)", "Level(low)"}, 
               {"Level(high)"}, {"Level(low)"}, 1),
        
        # Climb down from box
        Action("ClimbDown", {"Level(high)"}, {"Level(low)"}, {"Level(high)"}, 1),
        
        # Grab bananas (must be high at location B)
        Action("GrabBananas", {"At(B)", "Level(high)", "BananasAt(B)"}, 
               {"HasBananas"}, set(), 1)
    ]
    
    initial_state = {"At(A)", "Level(low)", "BoxAt(C)", "BananasAt(B)"}
    goal_state = {"HasBananas"}
    
    print(f"\nInitial state: {initial_state}")
    print(f"Goal state: {goal_state}")
    
    planner = GOAPPlanner(actions)
    plan, cost = planner.plan(initial_state, goal_state)
    print_plan(plan, cost, initial_state)

## Case2-Shakey-the-robot

In [6]:
def example_shakey_robot():
    """
    Shakey the Robot - First robot to use STRIPS planning (1966-1972).
    
    Scenario:
    - Robot moves between rooms
    - Robot can push boxes
    - Doors can be open or closed
    """
    print("\nEXAMPLE 2: Shakey the Robot")
    
    actions = [
        # Move between rooms (door must be open)
        Action("GoTo(Room1->Room2)", {"RobotIn(Room1)", "DoorOpen(Room1,Room2)"}, 
               {"RobotIn(Room2)"}, {"RobotIn(Room1)"}, 2),
        Action("GoTo(Room2->Room1)", {"RobotIn(Room2)", "DoorOpen(Room1,Room2)"}, 
               {"RobotIn(Room1)"}, {"RobotIn(Room2)"}, 2),
        Action("GoTo(Room2->Room3)", {"RobotIn(Room2)", "DoorOpen(Room2,Room3)"}, 
               {"RobotIn(Room3)"}, {"RobotIn(Room2)"}, 2),
        Action("GoTo(Room3->Room2)", {"RobotIn(Room3)", "DoorOpen(Room2,Room3)"}, 
               {"RobotIn(Room2)"}, {"RobotIn(Room3)"}, 2),
        
        # Open doors
        Action("OpenDoor(Room1,Room2)", {"RobotIn(Room1)", "DoorClosed(Room1,Room2)"}, 
               {"DoorOpen(Room1,Room2)"}, {"DoorClosed(Room1,Room2)"}, 1),
        Action("OpenDoor(Room2,Room3)", {"RobotIn(Room2)", "DoorClosed(Room2,Room3)"}, 
               {"DoorOpen(Room2,Room3)"}, {"DoorClosed(Room2,Room3)"}, 1),
        
        # Push box between rooms
        Action("PushBox(Room1->Room2)", 
               {"RobotIn(Room1)", "BoxIn(Room1)", "DoorOpen(Room1,Room2)"}, 
               {"RobotIn(Room2)", "BoxIn(Room2)"}, 
               {"RobotIn(Room1)", "BoxIn(Room1)"}, 3),
        Action("PushBox(Room2->Room3)", 
               {"RobotIn(Room2)", "BoxIn(Room2)", "DoorOpen(Room2,Room3)"}, 
               {"RobotIn(Room3)", "BoxIn(Room3)"}, 
               {"RobotIn(Room2)", "BoxIn(Room2)"}, 3)
    ]
    
    initial_state = {
        "RobotIn(Room1)", "BoxIn(Room1)",
        "DoorClosed(Room1,Room2)", "DoorClosed(Room2,Room3)"
    }
    goal_state = {"BoxIn(Room3)"}
    
    print(f"\nInitial state: {initial_state}")
    print(f"Goal state: {goal_state}")
    
    planner = GOAPPlanner(actions)
    plan, cost = planner.plan(initial_state, goal_state)
    print_plan(plan, cost, initial_state)

## Case3-Game-AI

In [7]:
def example_game_ai():
    # Game AI example: NPC builds and lights a campfire.    
    # This demonstrates typical GOAP usage in games
    print("\nEXAMPLE 3: Game AI - Build Campfire")
    
    actions = [
        # Movement
        Action("GoToForest", set(), {"AtForest"}, {"AtCamp", "AtRiver"}, 2),
        Action("GoToCamp", set(), {"AtCamp"}, {"AtForest", "AtRiver"}, 2),
        Action("GoToRiver", set(), {"AtRiver"}, {"AtCamp", "AtForest"}, 2),
        
        # Resource gathering
        Action("ChopWood", {"AtForest", "HasAxe"}, {"HasWood"}, set(), 4),
        Action("GatherBranches", {"AtForest"}, {"HasWood"}, set(), 8),  # Slower without axe
        Action("FindFlint", {"AtRiver"}, {"HasFlint"}, set(), 3),
        Action("GetAxe", {"AtCamp", "AxeAvailable"}, {"HasAxe"}, {"AxeAvailable"}, 1),
        
        # Build campfire
        Action("BuildFirepit", {"AtCamp", "HasWood"}, {"FirepitReady"}, {"HasWood"}, 2),
        Action("LightFire", {"AtCamp", "FirepitReady", "HasFlint"}, {"FireLit"}, set(), 1)
    ]
    
    # Test 1: With axe available
    print("\n Test 1: Axe available")
    initial_state = {"AtCamp", "AxeAvailable"}
    goal_state = {"FireLit"}
    
    print(f"Initial state: {initial_state}")
    print(f"Goal state: {goal_state}")
    
    planner = GOAPPlanner(actions)
    plan, cost = planner.plan(initial_state, goal_state)
    print_plan(plan, cost, initial_state)
    
    # Test 2: Without axe (must gather branches manually)
    print("\n Test 2: No axe available")
    initial_state_no_axe = {"AtCamp"}
    
    print(f"Initial state: {initial_state_no_axe}")
    print(f"Goal state: {goal_state}")
    
    plan2, cost2 = planner.plan(initial_state_no_axe, goal_state)
    print_plan(plan2, cost2, initial_state_no_axe)

## Case4-Dynamic-action-generation

In [8]:
def example_dynamic_actions():
    # Demonstrate dynamic action generation.
    
    # Instead of writing each action manually, generate them programmatically.
    # Useful for large game worlds with many locations.
    print("\nEXAMPLE 4: Dynamic Action Generation")
    
    def create_movement_actions(locations):
        """Generate movement actions between all location pairs."""
        actions = []
        for from_loc in locations:
            for to_loc in locations:
                if from_loc != to_loc:
                    actions.append(Action(
                        name=f"Move({from_loc}->{to_loc})",
                        preconditions={f"At({from_loc})"},
                        add_effects={f"At({to_loc})"},
                        delete_effects={f"At({from_loc})"},
                        cost=1
                    ))
        return actions
    
    def create_pickup_action(item, location):
        """Generate pickup action for item at location."""
        return Action(
            name=f"PickUp({item})",
            preconditions={f"At({location})", f"{item}At({location})"},
            add_effects={f"Has({item})"},
            delete_effects={f"{item}At({location})"},
            cost=1
        )
    
    # Create world
    locations = ["Home", "Shop", "Park", "Library"]
    actions = create_movement_actions(locations)
    
    # Add item pickups
    actions.append(create_pickup_action("Book", "Library"))
    actions.append(create_pickup_action("Food", "Shop"))
    actions.append(create_pickup_action("Ball", "Park"))
    
    # Add special action
    actions.append(Action(
        name="HavePicnic",
        preconditions={"At(Park)", "Has(Food)", "Has(Ball)"},
        add_effects={"HadFun"},
        delete_effects={"Has(Food)"},
        cost=5
    ))
    
    print(f"\nGenerated {len(actions)} actions")
    
    initial_state = {"At(Home)", "BookAt(Library)", "FoodAt(Shop)", "BallAt(Park)"}
    goal_state = {"HadFun", "Has(Book)"}
    
    print(f"Initial state: {initial_state}")
    print(f"Goal state: {goal_state}")
    
    planner = GOAPPlanner(actions)
    plan, cost = planner.plan(initial_state, goal_state)
    print_plan(plan, cost, initial_state)

#### main-run

In [9]:
# Run all examples
example_monkey_and_bananas()
example_shakey_robot()
example_game_ai()
example_dynamic_actions()


EXAMPLE 1: Monkey and Bananas

Initial state: {'BoxAt(C)', 'Level(low)', 'At(A)', 'BananasAt(B)'}
Goal state: {'HasBananas'}

Plan found! Total cost: 5
Number of actions: 4

Actions:
  1. Move(A->C) (cost: 1)
  2. PushBox(C->B) (cost: 2)
  3. ClimbUp (cost: 1)
  4. GrabBananas (cost: 1)

Final state: {'Level(high)', 'At(B)', 'BoxAt(B)', 'HasBananas', 'BananasAt(B)'}

EXAMPLE 2: Shakey the Robot

Initial state: {'BoxIn(Room1)', 'DoorClosed(Room2,Room3)', 'DoorClosed(Room1,Room2)', 'RobotIn(Room1)'}
Goal state: {'BoxIn(Room3)'}

Plan found! Total cost: 8
Number of actions: 4

Actions:
  1. OpenDoor(Room1,Room2) (cost: 1)
  2. PushBox(Room1->Room2) (cost: 3)
  3. OpenDoor(Room2,Room3) (cost: 1)
  4. PushBox(Room2->Room3) (cost: 3)

Final state: {'BoxIn(Room3)', 'DoorOpen(Room1,Room2)', 'RobotIn(Room3)', 'DoorOpen(Room2,Room3)'}

EXAMPLE 3: Game AI - Build Campfire

 Test 1: Axe available
Initial state: {'AtCamp', 'AxeAvailable'}
Goal state: {'FireLit'}

Plan found! Total cost: 17
Number 

## How-it-works

**1. PriorityQueue class** is the "manager" of the A* search algorithm. Its job is to keep track of which states we need to visit
  next, ensuring we always explore the most promising state first.

   * Role: It manages the Open List in the A* algorithm.
   * Key Behavior: When you put items in (Enqueue), it doesn't just add them to the end. It inserts them in a specific
     order based on their priority (cost).
       * Low Priority Value = High Importance: In pathfinding, a lower "cost" (f-score) is better. So, the item with the
         lowest number sits at index 0, ready to be processed first.
   * Key Methods:
       * `Enqueue(value, priority)`: Inserts a state into the queue while keeping the list sorted by priority.
       * `Dequeue()`: Removes and returns the item with the lowest priority value (the best candidate).
       * `Contains(value)`: Checks if a specific state is already waiting in the queue (prevents duplicates).

---

**2. Action class** defines the "rules" of the world. In STRIPS planning, it must define exactly when an action can happen and what changes it causes.

   * Components:
       * Preconditions: What must be true before you can do this? (e.g., to "Unlock Door", you must have "Key").
       * Add Effects: What becomes true after you do this? (e.g., "Door is Unlocked").
       * Delete Effects: What is no longer true after you do this? (e.g., "Key" might be gone if it breaks, or "Door is
         Locked" is removed).
       * Cost: How "expensive" is this action? (Used by A* to find the shortest/cheapest path).
   * Key Methods:
       * `is_applicable(state)`: Checks if the current world state contains all the Preconditions. If even one is missing,
         the action cannot be performed.
       * `apply(state)`: Creates a new state by taking the current state, removing the Delete Effects, and adding the Add
         Effects.

---

**3. GOAPPlanner class** is the "brain" that runs the A* algorithm. It uses the Action rules to figure out how to get from point A
  (Initial State) to point B (Goal State).

   * `heuristic(state, goal)`:
       * This functions estimates how close we are to the goal.
       * Your implementation counts how many goal conditions are missing.
       * Example: If the goal is {"HasBananas", "Safe"} and we only have {"Safe"}, the distance is 1.
   * `plan(initial_state, goal_state)`:
       * This is the main A* loop.
       * `g_score`: Tracks the actual cost from the start to the current state.
       * `f_score`: Estimated total cost (g_score + heuristic).
       * It pulls the best state from the PriorityQueue.
       * It finds all valid actions (get_applicable_actions).
       * It applies those actions to generate new states (neighbors).
       * If a new state is closer/cheaper, it records it and adds it to the queue.
   * `_reconstruct_plan`:
       * Once the goal is reached, this function simply backtracks. It looks at the came_from dictionary to see which
         action got us to the goal, then which action got us to that state, all the way back to the start.

---

**How the System Works Together**

   1. Setup: You define the Initial State (e.g., {"At(Home)", "Hungry"}) and the Goal State (e.g., {"NotHungry"}).
   2. The Loop: The GOAPPlanner looks at the Initial State.
   3. Expansion: It asks: "What actions can I perform right now?" (checking is_applicable on all Actions).
       * Maybe you can "EatFood" (if you have food) or "GoToStore".
   4. Simulation: It simulates these actions using apply() to create theoretical New States.
       * State A: {"At(Home)", "NotHungry"} (Cost: 1)
       * State B: {"At(Store)", "Hungry"} (Cost: 2)
   5. Selection: It uses the PriorityQueue to pick the state with the lowest f_score.
       * If State A satisfies the Goal, it stops!
       * If not, it repeats the process from that new state.
   6. Result: It returns the list of actions that successfully transformed the Initial State into the Goal State.